#### 라이브러리 정의

In [ ]:
# - 데이터를 읽기 위한 라이브러리
import pandas as pd

# - 파이썬에서 사용되는 기본 시각화 라이브러리
import matplotlib.pyplot as plt

import numpy as np
from sklearn.cluster import KMeans
import requests
import time
import io

from scipy.interpolate import RBFInterpolator

from math import radians, sin, cos, sqrt, atan2

### 경고(오류는 아님) 메시지 없애기
# - 사이킷런 버전에 따라 오류가 아니니 안내(경고)메시지가 자주 나타남
# - 안내(경고) 메시지 없이 실행할 수 있도록 처리
from sklearn import set_config
set_config(display="text")

# - 한글처리
plt.rc("font", family="Malgun Gothic")

# - 마이너스 기호 깨짐 처리
plt.rcParams["axes.unicode_minus"] = False

#### 기상 데이터 수집

In [2]:
# 사우스웨일스주 경계 좌표
lat_range = (-37.5, -29.0)
lon_range = (150.0, 159.0)

# 0.1도 간격 격자 생성 (약 10km)
lat_points = np.arange(lat_range[0], lat_range[1], 0.1)
lon_points = np.arange(lon_range[0], lon_range[1], 0.1)
grid = [(lat, lon) for lat in lat_points for lon in lon_points]
grid_df = pd.DataFrame(grid, columns=["latitude", "longitude"])

# K-Means 클러스터링: 캘리포니아 좌표를 50개의 대표 좌표로 압축
# 목적: API 요청 횟수를 줄이고 데이터 크기를 줄이면서 지역적 대표성을 유지
kmeans = KMeans(n_clusters=50, random_state=42)
grid_df["cluster"] = kmeans.fit_predict(grid_df[["latitude", "longitude"]])
cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=["latitude", "longitude"])

# 클러스터링 결과 확인
print("클러스터 중심 좌표 (50개 대표 좌표):")
print(cluster_centers.head())

# 클러스터 중심에서 기상 데이터 수집
start_date = "20150101"
end_date = "20250228"

def get_skiprows(response_text):
    lines = response_text.splitlines()
    skiprows = 0
    for line in lines:
        if line.strip() == "YEAR,DOY,T2M,WS2M,RH2M,PRECTOTCORR":
            break
        skiprows += 1
    return skiprows

def get_weather_data(latitude, longitude):
    url = f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,WS2M,RH2M,PRECTOTCORR&community=AG&longitude={longitude}&latitude={latitude}&start={start_date}&end={end_date}&format=CSV"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        if response.status_code == 200:
            # 응답 데이터 구조 확인 (디버깅용)
            print(f"응답 데이터 상단 15줄 (위도: {latitude}, 경도: {longitude}):")
            print("\n".join(response.text.splitlines()[:15]))
            # 동적으로 skiprows 계산
            skiprows = get_skiprows(response.text)
            return pd.read_csv(io.StringIO(response.text), skiprows=skiprows, header=0)
        else:
            print(f"데이터 요청 실패! 상태 코드: {response.status_code}, 위도: {latitude}, 경도: {longitude}")
            return None
    except requests.exceptions.RequestException as e:
        print(f"요청 오류 발생: {e}, 위도: {latitude}, 경도: {longitude}")
        return None

weather_data_list = []
for idx, row in cluster_centers.iterrows():
    print(f"요청 중: 클러스터 {idx} (위도 {row['latitude']}, 경도 {row['longitude']})")
    weather_data = get_weather_data(row["latitude"], row["longitude"])
    if weather_data is not None:
        weather_data["latitude"] = row["latitude"]
        weather_data["longitude"] = row["longitude"]
        weather_data_list.append(weather_data)
    time.sleep(5)  # API 요청 간 대기

# 데이터프레임 병합
weather_df = pd.concat(weather_data_list, ignore_index=True)

# acq_date 생성
weather_df["acq_date"] = pd.to_datetime(weather_df["YEAR"].astype(str) + weather_df["DOY"].astype(str), format="%Y%j")

# 결과 출력
print("\n기상 데이터 정보:")
print(weather_df.info())
print(weather_df.head(5))

# 저장
weather_df.to_csv("./data/csv/weather_data.csv", index=False)

클러스터 중심 좌표 (50개 대표 좌표):
    latitude   longitude
0 -34.292105  157.235526
1 -30.899237  153.514504
2 -36.982069  154.067586
3 -29.597076  155.324561
4 -35.598052  151.117532
요청 중: 클러스터 0 (위도 -34.29210526315785, 경도 157.23552631578906)
응답 데이터 상단 15줄 (위도: -34.29210526315785, 경도: 157.23552631578906):
-BEGIN HEADER-
NASA/POWER Source Native Resolution Daily Data 
Dates (month/day/year): 01/01/2015 through 02/28/2025 in LST
Location: latitude  -34.2921   longitude 157.2355 
elevation from MERRA-2: Average for 0.5 x 0.625 degree lat/lon region = 0.0 meters
The value for missing source data that cannot be computed or is outside of the sources availability range: -999 
parameter(s): 
T2M             MERRA-2 Temperature at 2 Meters (C) 
WS2M            MERRA-2 Wind Speed at 2 Meters (m/s) 
RH2M            MERRA-2 Relative Humidity at 2 Meters (%) 
PRECTOTCORR     MERRA-2 Precipitation Corrected (mm/day) 
-END HEADER-
YEAR,DOY,T2M,WS2M,RH2M,PRECTOTCORR
2015,1,21.95,4.15,71.47,0.03
2015,2,22.51,4.

#### 산불 데이터와 기상데이터를 더해서 하나의 CSV 파일로 생성

In [3]:
# Haversine 공식으로 거리 계산
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구 반지름 (km)
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

# IDW(역거리 가중 보간) 함수
def idw_interpolation(row, weather_df, weather_vars):
    # 동일 날짜의 기상 데이터만 필터링
    weather_subset = weather_df[weather_df["acq_date"] == row["acq_date"]]
    if weather_subset.empty:
        return pd.Series([None] * len(weather_vars), index=weather_vars)
    
    distances = weather_subset.apply(
        lambda w: haversine(row["latitude"], row["longitude"], w["latitude"], w["longitude"]), axis=1
    )
    # 거리가 0인 경우를 방지하기 위해 최소값 설정
    distances = distances.replace(0, 1e-10)
    weights = 1 / (distances ** 2)
    weighted_sum = (weather_subset[weather_vars].multiply(weights, axis=0)).sum()
    return weighted_sum / weights.sum()

# 1. 산불 데이터 로드
wildfire_dfs = {
    'aS1': pd.read_csv("./data/csv/fire_archive_SV-C2_591013.csv"),
    'nS1': pd.read_csv("./data/csv/fire_nrt_SV-C2_591013.csv"),
}

# 2. 날씨 데이터 로드
weather_df = pd.read_csv("./data/csv/weather_data.csv")

# 3. 성능 최적화: dtypes 최적화
# 메모리 사용량을 줄이기 위해 float32 사용
for df in wildfire_dfs.values():
    df["latitude"] = df["latitude"].astype("float32")
    df["longitude"] = df["longitude"].astype("float32")

weather_df["latitude"] = weather_df["latitude"].astype("float32")
weather_df["longitude"] = weather_df["longitude"].astype("float32")
weather_df["T2M"] = weather_df["T2M"].astype("float32")
weather_df["WS2M"] = weather_df["WS2M"].astype("float32")
weather_df["RH2M"] = weather_df["RH2M"].astype("float32")
weather_df["PRECTOTCORR"] = weather_df["PRECTOTCORR"].astype("float32")

# 4. 산불 데이터 처리
for df in wildfire_dfs.values():
    df["acq_date"] = pd.to_datetime(df["acq_date"])

wildfire_df = pd.concat(wildfire_dfs.values(), ignore_index=True)

# 5. 기상 데이터 처리
weather_df["acq_date"] = pd.to_datetime(weather_df["YEAR"].astype(str) + weather_df["DOY"].astype(str), format="%Y%j")

# 성능 최적화: 병합 전 필터링 (2025-03-01 이전 데이터만 사용)
cutoff_date = pd.Timestamp("2025-03-01")
weather_df = weather_df[weather_df["acq_date"] <= cutoff_date]
wildfire_df = wildfire_df[wildfire_df["acq_date"] <= cutoff_date]

# 6. 기상 데이터 보간 (클러스터별로 RBF 보간)
# 클러스터 식별을 위해 latitude와 longitude 조합으로 고유 키 생성
weather_df["cluster_id"] = weather_df["latitude"].astype(str) + "_" + weather_df["longitude"].astype(str)

# 보간 전 결측치 확인
print("보간 전 결측치:")
print(weather_df[["T2M", "WS2M", "RH2M", "PRECTOTCORR"]].isnull().sum())

# RBF 보간 적용
weather_vars = ["T2M", "WS2M", "RH2M", "PRECTOTCORR"]
for cluster_id in weather_df["cluster_id"].unique():
    cluster_data = weather_df[weather_df["cluster_id"] == cluster_id].copy()
    if cluster_data[weather_vars].isnull().any().any():
        # 시간 축 (acq_date를 숫자로 변환)
        time_points = (cluster_data["acq_date"] - cluster_data["acq_date"].min()).dt.days.values
        # 결측치가 없는 데이터만 사용
        for var in weather_vars:
            mask = ~cluster_data[var].isnull()
            if mask.sum() > 1:  # 최소 2개 이상의 데이터 포인트 필요
                rbf = RBFInterpolator(
                    time_points[mask].reshape(-1, 1),
                    cluster_data[var][mask],
                    kernel="thin_plate_spline"
                )
                # 결측치가 있는 위치에 대해 보간
                missing_mask = cluster_data[var].isnull()
                if missing_mask.any():
                    interpolated_values = rbf(time_points[missing_mask].reshape(-1, 1))
                    cluster_data.loc[missing_mask, var] = interpolated_values
        # 보간된 데이터로 업데이트
        weather_df.loc[weather_df["cluster_id"] == cluster_id, weather_vars] = cluster_data[weather_vars]

# 보간 후 결측치 확인
print("\nRBF 보간 후 결측치:")
print(weather_df[["T2M", "WS2M", "RH2M", "PRECTOTCORR"]].isnull().sum())

# 7. IDW 보간으로 기상 데이터 매핑
wildfire_df[weather_vars] = wildfire_df.apply(
    lambda row: idw_interpolation(row, weather_df, weather_vars), axis=1
)

# 병합된 데이터프레임 생성
merged_df = wildfire_df.copy()

# 병합 후 결측치 확인 (기상 데이터 매핑 검증)
print("\nIDW 적용 후 결측치:")
print(merged_df[weather_vars].isnull().sum())

# 8. 결측치 처리: 결측치가 있는 행은 선형 보간으로 채우기
for var in weather_vars:
    merged_df[var] = merged_df[var].interpolate(method='linear')

# 결측치 처리 후 확인
print("\n결측치 처리 후 결측치:")
print(merged_df[weather_vars].isnull().sum())

# 9. 2025년 3월 1일 이후 데이터 삭제 (필터링은 이미 적용됨, datetime 생성만 진행)
if "acq_time" in merged_df.columns:
    merged_df["datetime"] = pd.to_datetime(
        merged_df["acq_date"].astype(str) + " " + merged_df["acq_time"].astype(str).str.zfill(4),
        format="%Y-%m-%d %H%M"
    )
else:
    merged_df["datetime"] = merged_df["acq_date"]

# 10. 필요 없는 컬럼 삭제
columns_to_drop = ["acq_time"]
merged_df.drop(columns=columns_to_drop, inplace=True, errors='ignore')

# 11. 결과 출력
print("\n병합된 데이터 정보:")
print(merged_df.info())
print(merged_df.head(5))

보간 전 결측치:
T2M            0
WS2M           0
RH2M           0
PRECTOTCORR    0
dtype: int64

RBF 보간 후 결측치:
T2M            0
WS2M           0
RH2M           0
PRECTOTCORR    0
dtype: int64

IDW 적용 후 결측치:
T2M            0
WS2M           0
RH2M           0
PRECTOTCORR    0
dtype: int64

결측치 처리 후 결측치:
T2M            0
WS2M           0
RH2M           0
PRECTOTCORR    0
dtype: int64

병합된 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1181014 entries, 0 to 1181013
Data columns (total 19 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   latitude     1181014 non-null  float32       
 1   longitude    1181014 non-null  float32       
 2   brightness   1181014 non-null  float64       
 3   scan         1181014 non-null  float64       
 4   track        1181014 non-null  float64       
 5   acq_date     1181014 non-null  datetime64[ns]
 6   satellite    1181014 non-null  object        
 7   instrument   1181014 non-null  

In [4]:
# 12. 병합된 데이터 저장
merged_df.to_csv("./data/csv/merged_wildfire_weather.csv", index=False)

In [5]:
# 병합된 데이터 정보 출력
print("\n병합된 데이터 정보:")
print(merged_df.info())

# 데이터 일부 출력 (병합이 제대로 되었는지 확인)
print("\n병합된 데이터 미리보기:")
print(merged_df.head(5))


병합된 데이터 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1181014 entries, 0 to 1181013
Data columns (total 19 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   latitude     1181014 non-null  float32       
 1   longitude    1181014 non-null  float32       
 2   brightness   1181014 non-null  float64       
 3   scan         1181014 non-null  float64       
 4   track        1181014 non-null  float64       
 5   acq_date     1181014 non-null  datetime64[ns]
 6   satellite    1181014 non-null  object        
 7   instrument   1181014 non-null  object        
 8   confidence   1181014 non-null  object        
 9   version      1181014 non-null  object        
 10  bright_t31   1181014 non-null  float64       
 11  frp          1181014 non-null  float64       
 12  daynight     1181014 non-null  object        
 13  type         1175950 non-null  float64       
 14  T2M          1181014 non-null  float64       
 15  WS

In [6]:
# 결측치 확인
print("\n병합 후 결측치 확인:")
print(merged_df[weather_vars].isnull().sum())


병합 후 결측치 확인:
T2M            0
WS2M           0
RH2M           0
PRECTOTCORR    0
dtype: int64
